In [24]:
import torch
from torchvision import datasets, transforms
batch_size = 200
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms),
    batch_size=batch_size, shuffle=True)

In [25]:
import torch.nn.functional as F
from torch.nn import Module, Sequential, Linear, LeakyReLU
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import time
#网络结构
class MLP(Module):
    def __init__(self):
        super(MLP,self).__init__()
        
        self.model= Sequential(
            Linear(784,200),
            LeakyReLU(inplace=True),
            Linear(200,200),
            LeakyReLU(inplace=True),
            Linear(200,10),
            LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.model(x)

In [26]:
batch_size = 200
learning_rate = 0.001
#整个数据集
epochs = 10
# 定义GPU设备
# device = torch.device('cuda:0')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model放到GPU
net = MLP().to(device)
#选择一个优化器，指定需要优化的参数，以及学习率
optimizer =torch.optim.SGD(net.parameters(),lr=learning_rate)

In [27]:
for epoch in range(epochs):
    for batch_idx,(data,target)in enumerate(train_loader):
        # data转换维度为[200,784],target的维度为[200]
        data= data.view(-1,28 *28)
        #将data和target放到GPU
        data, target= data.to(device),target.to(device)
        # data为输入，net()直接执行forward
        # 跑一次网络，得到z,维度为[200,10],200是batch size,10是类别
        #由于net在GPU，data也在GPU，计算出的z就在GPU
        # 调用net(data)的时候相当于调用Module类的 cal1 方法
        z= net(data)
        #将loss放到GPU
        loss =F.cross_entropy(z,target).to(device)
        #每次迭代前将梯度置0
        optimizer.zero_grad()
        #反向传播，计算梯度
        loss.backward()
        #相当于执行w=w- dw，也就是更新权值
        optimizer.step()
        if batch_idx % 100 == 0:
            print(loss)

tensor(2.3091, grad_fn=<NllLossBackward0>)
tensor(2.2879, grad_fn=<NllLossBackward0>)
tensor(2.2699, grad_fn=<NllLossBackward0>)
tensor(2.2618, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 